In [ ]:
import torch
import torch.nn as nn

inputs = torch.tensor(
    [[0.1, 0.2, 0.3],
    [0.2, 0.2, 0.2],
    [0.3, 0.3, 0.3],
    [0.4, 0.4, 0.4],
    [0.5, 0.5, 0.5],
    [0.6, 0.6, 0.6],]
    )

print(inputs)

print(inputs.shape)

class SimpleSelfAttention(nn.parameter):
    super.__init__(self, w_q, w_k, w_v):
    self.w_q = torch.nn.parameter(torch.rand(d_in, d_out))
    self.w_k = torch.nn.parameter(torch.rand(d_in, d_out))
    self.w_v = torch.nn.parameter(torch.rand(d_in, d_out))

    def forward(arg):
        query = w_q @ inputs
        values = w_k @ inputs
        context_vec = query / d_k ** 0.5

    


SyntaxError: invalid syntax (3398950686.py, line 18)